In [1]:
import selenium
from selenium import webdriver


from bs4 import BeautifulSoup
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import time

In [17]:
mangoData=pd.DataFrame(columns=["title","location","estimation","viewCount","reviewCount"])
path= r"C:\Users\cxz01\OneDrive\바탕 화면\soongsil university\데이터사이언스\팀프로젝트\DS_Univ_5\web_crawling\chromedriver_win32\chromedriver.exe"
browser = webdriver.Chrome(path)
areaList = ["가로수길","강남역","강동구","개포","수서","일원","관악구","교대","서초","구로구","금천구","논현동","대치동","도곡동","동작","사당","등촌","강서","목동","양천","방배","반포","잠원","방이동","삼성동","서래마을","송파",'가락','신사','압구정','신첨','잠실','양재동','여의도','역삼','선릉','영등포구','청담동','건대','군자','광진','광화문','노원구','대학로','혜화','동대문구','동부이촌동','마포','공덕','명동','남산','삼청','인사','상암','성산','서대문구','성북구','수유','도봉','강북','시청','남대문','신촌','이대','연남동','왕십리','성동','용산','숙대','은평구','이태원','한남동','종로','중구','중랑구','합정','망원','홍대']
breakSign = False

#검색 총 page 개수: 50
for area in areaList:
    for page in range(1,51):
        if breakSign:
            breakSign = False
            break
        mangoplateURL=f"https://www.mangoplate.com/search/{area}?keyword={area}&page={page}"
        browser.get(mangoplateURL)
        time.sleep(2)
        pageSource = BeautifulSoup(browser.page_source)
        infoList=pageSource.find_all("div", attrs = {"class":"info"})
        
        for info in infoList[0:20]:
            title = info.find("h2", attrs = {"class" : "title"}).get_text()
            detailURL = info.find("a")["href"]
            try:
                estimation = info.find("strong", attrs = {"class" : "point search_point"}).get_text()
            except:
                estimation = info.find("strong", attrs = {"class" : "point search_point expected"}).get_text()
            viewCount = info.find("span", attrs = {"class" : "view_count"}).get_text()
            reviewCount = info.find("span", attrs = {"class" : "review_count"}).get_text()

            title = title.strip()
            detailURL = "https://www.mangoplate.com" + detailURL
            try:
                estimation = float(estimation)
            except:
                breakSign = True
                break
            viewCount = float("".join(viewCount.split(",")))
            reviewCount = float("".join(reviewCount.split(",")))

            browser.get(detailURL)
            time.sleep(1)
            pageSource = BeautifulSoup(browser.page_source)
            
            # table_rows를 정의합니다
            table_rows = pageSource.find_all('tr')

            foodType = None
            price = None
            runTime = None
            location = None
            
            # 모든 행을 순회합니다
            for row in table_rows:
                ths = row.find_all('th')
                tds = row.find_all('td')

                for i, th in enumerate(ths):
                    if "음식 종류" in th.text:
                        foodType = tds[i].text.strip()   # 해당하는 td의 텍스트를 추출합니다
                    if "가격대" in th.text:
                        price = tds[i].text.strip() # 해당하는 td의 텍스트를 추출합니다
                    if "영업시간" in th.text:
                        runTime = tds[i].text.strip() # 해당하는 td의 텍스트를 추출합니다                
            
            location = pageSource.find("span", attrs = {"class" : "Restaurant__InfoAddress--Text"}).get_text()
            
            newData = {"title" : [title], "location" : [location], "estimation" : [estimation], "viewCount": [viewCount], "reviewCount" : [reviewCount], "foodType" : [foodType], "price" : [price], "runTime" : [runTime]}
            newData = pd.DataFrame(newData)
            mangoData = pd.concat((mangoData, newData)).reset_index(drop = True)


C:\Users\cxz01\AppData\Local\Temp\ipykernel_16548\3010246121.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(path)


WebDriverException: Message: disconnected: not connected to DevTools
  (failed to check if window was closed: disconnected: not connected to DevTools)
  (Session info: chrome=113.0.5672.127)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x00368893+48451]
	(No symbol) [0x002FB8A1]
	(No symbol) [0x00205058]
	(No symbol) [0x001F7BB6]
	(No symbol) [0x001F78D9]
	(No symbol) [0x00206460]
	(No symbol) [0x0025C038]
	(No symbol) [0x0024A0B6]
	(No symbol) [0x00227E08]
	(No symbol) [0x00228F2D]
	GetHandleVerifier [0x005C8E3A+2540266]
	GetHandleVerifier [0x00608959+2801161]
	GetHandleVerifier [0x0060295C+2776588]
	GetHandleVerifier [0x003F2280+612144]
	(No symbol) [0x00304F6C]
	(No symbol) [0x003011D8]
	(No symbol) [0x003012BB]
	(No symbol) [0x002F4857]
	BaseThreadInitThunk [0x76427D59+25]
	RtlInitializeExceptionChain [0x7783B74B+107]
	RtlClearBits [0x7783B6CF+191]


In [16]:
mangoData

,title,location,estimation,viewCount,reviewCount,foodType,price,runTime
0,플라워베이커리,서울시 강남구 신사동 517-5,3.5,11008.0,20.0,베이커리,만원 미만,10:00 - 22:00
1,온달집,서울시 강남구 신사동 514-4,3.5,22229.0,28.0,전통 주점 / 포차,만원-2만원,일-목: 17:00 - 05:00 금-토: 17:00 - 06:00
2,코다차야\n (가로수길점),서울시 강남구 신사동 548-4,3.5,15279.0,31.0,이자카야 / 오뎅 / 꼬치,만원-2만원,17:00 - 03:00
3,신사양꼬치,서울시 강남구 신사동 515-10 2F,3.5,26457.0,21.0,기타 중식,2만원-3만원,None
4,미드타운,서울시 강남구 신사동 535-3,3.5,3309.0,9.0,브런치 / 버거 / 샌드위치,만원-2만원,None


In [ ]:
mangoData.to_csv('mangoData.csv', index=False, encoding='utf-8')

In [ ]:
geoData = pd.read_excel(r"C:\Users\cxz01\OneDrive\바탕 화면\soongsil university\데이터사이언스\팀프로젝트\DS_Univ_5\web_crawling\mango\geoData.xlsx")
geoData = geoData[geoData["시도"] == "서울특별시"]
geoData = geoData.drop_duplicates(subset = ["시군구", "읍면동"])
geoData["geo"] = geoData["시도"] + " " + geoData["시군구"] + " " + geoData["읍면동"]
geoData


In [ ]:
mapData = pd.read_csv(r"C:\Users\cxz01\OneDrive\바탕 화면\soongsil university\데이터사이언스\팀프로젝트\DS_Univ_5\web_crawling\mango\mangoData.csv")
mapData["geo_1"] = "서울특별시"
mapData["geo_2"] = mapData["location"].str.split(" ").str[1]
mapData["geo_3"] = mapData["location"].str.split(" ").str[2]
mapData["geo"] = mapData["geo_1"] + " " + mapData["geo_2"] + " " + mapData["geo_3"]

mapData = pd.merge(left = mapData, right = geoData[["위도", "경도", "geo"]], how = "left", left_on = "geo", right_on = "geo")
mapData = mapData.dropna(subset = ["위도","경도"]).reset_index(drop=True)
mapData

In [ ]:
mapData.to_csv('mangoMapData.csv', index=False, encoding='utf-8')

In [ ]:
mapData = mapData[['estimation','reviewCount','위도','경도']]

In [ ]:
mapData

In [ ]:
mapData.to_csv('mangoMapData2.csv', index=False, encoding='utf-8')